<a href="https://colab.research.google.com/github/nthammadi-uncc/MultiEffects_AirPollutants/blob/main/Notebooks/AP_Data_Cleanup_and_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleanup and Preprocessing for Bayesian Multi Effect model Analysis of Air pollutants through Electricity usage across all states of America

#### Install libraries

In [1]:
#run both R and python 
%load_ext rpy2.ipython

In [2]:
%%R
library(readr)
library(dplyr)
library(tidyverse)
library(magrittr)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ tidyr   1.2.0     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: 
Attaching package: ‘magrittr’


R[write to console]: The following object is masked from ‘package:purrr’:

    set_names


R[write to console]: The following object is masked from ‘package:tidyr’:

    extract




#### R version

In [3]:
%%R
R.version.string

[1] "R version 4.2.0 (2022-04-22)"


#### Data files from Github

In [4]:
%%R
#set up data urls from github
parent_url<-"https://raw.githubusercontent.com/nthammadi-uncc/electricity_usage_analysis/main/Data/"
electric_price_url <- paste(parent_url,"avgprice_annual.csv", sep="")
#annual_customers_url <- paste(parent_url,"customers_annual.csv", sep="")
annual_emission_url <- paste(parent_url,"emission_annual.csv", sep="")
#annual_revenue_url <- paste(parent_url,"revenue_annual.csv", sep="")
monthly_consumption_url <- paste(parent_url,"consumption_monthly.csv", sep="")
monthly_generation_url <- paste(parent_url,"generation_monthly.csv", sep="")
#median_household_income_url <- paste(parent_url,"median_household_income.csv", sep="")
presidential_election_url <- paste(parent_url,"presidential_election_results.csv", sep="")
state_regions_url <- "https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv"

#### Load CSV files and perform Feature Engineering and Data Pre-processings

In [5]:
%%R
print("---------- Electricity price - cents per kilo watt hour ----------")
#Electricity price - cents per kilo watt hour
all_price_df <- read_csv(url(electric_price_url))
#convert column names to R-safe column names
names(all_price_df) <- make.names(names(all_price_df))
#only select "Total Electric Industry" industry sector and records of the year 2020
all_price_df <- all_price_df %>% filter(Industry.Sector.Category == "Total Electric Industry") %>% select(Year,State,Total)
#convert cents per KWh to dollars per MWh
#1 MWh = 1000 KWh
#1 dollar = 100 cents
all_price_df$PRICE.DOLLAR_MWh <- (all_price_df$Total * 1000) / 100
#rename columns
names(all_price_df) <- c('YEAR', 'STATE', 'PRICE.CENTS_KWh','PRICE.DOLLAR_MWh')
#only select records of the year 2020
price_df <- all_price_df %>% filter(YEAR == 2020) %>% select(YEAR,STATE,PRICE.DOLLAR_MWh)
#dimensions of the dataframe will be [52 - rows, 3 columns]. This includes DC and US-Total
dim(price_df)

print("---------- Electricity Generation by type(Sustainable,Not Sustainable) measured in Mega Watt hours ----------")
#Electricity Generation by type(Sustainable,Not Sustainable)
all_generation_df <- read_csv(url(monthly_generation_url))
#convert column names to R-safe column names
names(all_generation_df) <- make.names(names(all_generation_df))
#there are multiple energy sources termed under "Other". Only select those that can be categorized by type(Sustainable,Not Sustainable)
en_sources <- c("Coal","Geothermal","Hydroelectric Conventional","Natural Gas","Nuclear","Other Biomass","Other Gases","Petroleum","Pumped Storage","Solar Thermal and Photovoltaic","Wind","Wood and Wood Derived Fuels")
#only select "Total Electric Industry" type of producer and energy sources mentioned above
all_generation_df <- all_generation_df %>% filter(TYPE_OF_PRODUCER == "Total Electric Power Industry" & ENERGY_SOURCE %in% en_sources) %>% select(,-TYPE_OF_PRODUCER) 
#Data is distributed on monthly basis. Group by year, state and energy source and summarize on mean of GENERATION_MWh
all_generation_df <- all_generation_df %>% group_by(across(all_of(c("YEAR","STATE","ENERGY_SOURCE")))) %>% summarize(GENERATION_MWh =  mean(GENERATION_MWh, na.rm=TRUE))
#categorize energy source by type(Sustainable,Not Sustainable)
all_generation_df$ENERGY.SOURCE <- ifelse(all_generation_df$ENERGY_SOURCE %in% c("Coal","Petroleum","Other Gases","Wood and Wood Derived Fuels"), "Not Sustainable","Sustainable")
#group by year, state and type of energy source and summarize by adding the GENERATION_MWh of similar types
all_generation_df <- all_generation_df %>% group_by(across(all_of(c("YEAR","STATE","ENERGY.SOURCE")))) %>% summarize(GENERATION_MWh =  sum(GENERATION_MWh, na.rm=TRUE)) %>% ungroup()
#convert Generation MWh to million MWh
all_generation_df$GENERATION_MWh <- round(all_generation_df$GENERATION_MWh/(10^6),4)
#rename column
names(all_generation_df)[names(all_generation_df) == 'GENERATION_MWh'] <- 'GENERATION_MM.MWh'
#only store the records of year 2020 for further analysis
generation_df <- all_generation_df %>% filter(YEAR == 2020)
#dimensions of the dataframe will be [104 - rows, 4 columns], 52 rows for each type(Sustainable,Not Sustainable). This includes DC and US-Total
dim(generation_df)

print("---------- Electricity Consumption by type(Sustainable,Not Sustainable) measured in Mega Watt hours ----------")
#Electricity Consumption by type(Sustainable,Not Sustainable)
all_consumption_df <- read_csv(url(monthly_consumption_url))
#convert column names to R-safe column names
names(all_consumption_df) <- make.names(names(all_consumption_df))
#only select "Total Electric Power Industry" type of producer
all_consumption_df <- all_consumption_df %>% filter(TYPE_OF_PRODUCER == "Total Electric Power Industry") %>% select(,-TYPE_OF_PRODUCER) 
#Data is distributed on monthly basis. Group by year, state and energy source and summarize on mean of CONSUMPTION
all_consumption_df <- all_consumption_df %>% group_by(across(all_of(c("YEAR","STATE","ENERGY_SOURCE")))) %>% summarize(CONSUMPTION =  mean(CONSUMPTION, na.rm=TRUE)) %>% ungroup()
#categorize energy source by type(Sustainable,Not Sustainable). Other Gases (Billion Btu) includes blast furnace gas, and other manufactured and waste gases derived from fossil fuels.
all_consumption_df$ENERGY.SOURCE <- ifelse(all_consumption_df$ENERGY_SOURCE %in% c("Coal (Short Tons)","Petroleum (Barrels)","Other Gases (Billion Btu)"), "Not Sustainable","Sustainable")

#Each energy source is measured in a different unit of measure:
#   Coal - Short tons
#   Petroleum - Barrels
#   Natural gas - Mcf
#   Geothermal and other gases - Billion Btu

# To convert each of these into a common unit - Mega Watt hour, the next set of calculations are performed
# We will first separate each energy source by its unit into a separate dataframe and finally combine them all into one
coal_df <- all_consumption_df%>% filter(ENERGY_SOURCE == "Coal (Short Tons)")
petroleum_df <- all_consumption_df%>% filter(ENERGY_SOURCE == "Petroleum (Barrels)")
natural_gas_df <- all_consumption_df%>% filter(ENERGY_SOURCE == "Natural Gas (Mcf)")
geothermal_other_df <- all_consumption_df%>% filter(ENERGY_SOURCE == "Geothermal (Billion Btu)" | ENERGY_SOURCE == "Other Gases (Billion Btu)")

#convert Coal (Short Tons) to Mega Watt Hour
#1 short ton = 2000 pounds,  1.13 pounds generates 1 Kilo Watt Hour, 1000 kWh = 1MWh
coal_df$CONSUMPTION_MWh <- (coal_df$CONSUMPTION * 2000 * 1.13)/1000

#convert Petroleum (Barrels) to Mega Watt Hour
#1 barrel = 42 gallons,   0.08 gallons generates 1 Kilo Watt Hour, 1000 kWh = 1MWh
petroleum_df$CONSUMPTION_MWh <- (petroleum_df$CONSUMPTION * 42 * 0.08)/1000

#convert Natural Gas (Mcf) to Mega Watt Hour
#1 Mcf = 1000 cubic feet,   7.43 gallons generates 1 Kilo Watt Hour, 1000 kWh = 1MWh
natural_gas_df$CONSUMPTION_MWh <- (natural_gas_df$CONSUMPTION * 1000 * 7.43)/1000

#convert Geothermal and Other Gases in Billion Btu to Mega Watt Hour
#1 Billion = 10^9, 0.341296928 * 10^-5 Billion Btu generates 1 Kilo Watt Hour, 1000 kWh = 1MWh
geothermal_other_df$CONSUMPTION_MWh <- (geothermal_other_df$CONSUMPTION * 0.341296928 *(10^-5))/1000

#combine all dataframes into one dataframe
combined_consumption_df <- rbind(coal_df,petroleum_df,natural_gas_df,geothermal_other_df)

#Group by year, state and energy.source(Sustainable/Not sustainable) and summarize by adding the CONSUMPTION_MWh of similar types
combined_consumption_df <- combined_consumption_df %>% group_by(across(all_of(c("YEAR","STATE","ENERGY.SOURCE")))) %>% summarize(CONSUMPTION_MWh =  sum(CONSUMPTION_MWh, na.rm=TRUE)) %>% ungroup()

#convert Consumption MWh to million MWh
combined_consumption_df$CONSUMPTION_MWh <- round(combined_consumption_df$CONSUMPTION_MWh/(10^6),4)
#rename column
names(combined_consumption_df)[names(combined_consumption_df) == 'CONSUMPTION_MWh'] <- 'CONSUMPTION_MM.MWh'

#only store the records of previous year 2019 for further analysis
consumption_df <- combined_consumption_df %>% filter(YEAR == 2019)
#update column names
names(consumption_df)[names(consumption_df) == 'CONSUMPTION_MM.MWh'] <- 'CONSUMPTION_MM.MWh.tminus1'
names(consumption_df)[names(consumption_df) == 'YEAR'] <- 'YEAR.tminus1'
#add new column for current year
consumption_df$YEAR <- (consumption_df$YEAR.tminus1)+1
#dimensions of the dataframe will be [104 - rows, 4 columns], 52 rows for each type(Sustainable,Not Sustainable). This includes DC and US-Total
dim(consumption_df)

print("---------- Carbon Emissions by Energy type(Sustainable,Not Sustainable) measured in Metric Tons ----------")
#Annual Emissions by Energy type(Sustainable,Not Sustainable)
all_emission_df <- read_csv(url(annual_emission_url))
#convert column names to R-safe column names
names(all_emission_df) <- make.names(names(all_emission_df))
#there are multiple energy emission sources termed under "Other". Also we will discard "All Sources" as we will be observing at the individual level
#Only select those that can be categorized by type(Sustainable,Not Sustainable)
emi_sources <- c("Coal","Geothermal","Natural Gas","Other Biomass","Other Gases","Petroleum","Wood and Wood Derived Fuels")
#only select "Total Electric Power Industry" type of producer and energy emission sources mentioned above
#for this project, we will only look into Carbon emissions by different energy sources and discard Sulphur and Nitrogen emissions
all_emission_df <- all_emission_df %>% filter(Producer.Type == "Total Electric Power Industry" & Energy.Source %in% emi_sources) %>% select(Year, State, Energy.Source, CO2..Metric.Tons.)
#rename columns
names(all_emission_df) <- c('YEAR', 'STATE', 'ENERGY.SOURCE', 'CO2.EMISSION_METRIC.TONS')
#categorize energy source by type(Sustainable,Not Sustainable)
all_emission_df$EMISSION.SOURCE <- ifelse(all_emission_df$ENERGY.SOURCE %in% c("Coal","Petroleum","Other Gases","Wood and Wood Derived Fuels"), "Not Sustainable","Sustainable")
#group by year, state and type of emission source and summarize by adding the CO2..Metric.Tons. of similar types
all_emission_df <- all_emission_df %>% group_by(across(all_of(c("YEAR","STATE","EMISSION.SOURCE")))) %>% summarize(CO2.EMISSION_METRIC.TONS =  sum(CO2.EMISSION_METRIC.TONS, na.rm=TRUE)) %>% ungroup()

#convert CO2 emissions from metric tons to million metric tons
all_emission_df$CO2.EMISSION_METRIC.TONS <- round(all_emission_df$CO2.EMISSION_METRIC.TONS/(10^6),4)
#rename column
names(all_emission_df)[names(all_emission_df) == 'CO2.EMISSION_METRIC.TONS'] <- 'CO2.EMISSION_MM.METRIC.TONS'
#also rename emission source to energy source
names(all_emission_df)[names(all_emission_df) == 'EMISSION.SOURCE'] <- 'ENERGY.SOURCE'

#only store the records of year 2020 for further analysis
emission_df <- all_emission_df %>% filter(YEAR == 2020)
#dimensions of the dataframe will be [104 - rows, 4 columns], 52 rows for each type(Sustainable,Not Sustainable). This includes DC and US-Total
dim(emission_df)

print("---------- Election Results in all States ----------")
#We will use the final_electricity_data csv to get the results of elections since these are already arranged and calculated
#All other columns will be discarded
all_election_result_df <- read_csv(url(presidential_election_url))
#convert column names to R-safe column names
names(all_election_result_df) <- make.names(names(all_election_result_df))
#create a percentage of votes column for the percent of total votes the candidate received
all_election_result_df$PERCENT_VOTES=all_election_result_df$candidate_votes/all_election_result_df$total_votes
#seperate votes by party as columns so that each state has a percentage of votes per party 
all_election_result_df <- all_election_result_df %>% group_by(across(all_of(c("year","state_po","party_simplified")))) %>%   #group by year,state,party
                                                  summarize(PERCENT_VOTES =  mean(PERCENT_VOTES, na.rm=TRUE)) %>%     #summarize by mean of percent votes
                                                  mutate(idx= row_number()) %>%     #mutate as data frame with id
                                                  spread(party_simplified,PERCENT_VOTES) %>%    #spread the party_simplified as columns and assign the row value as percent votes
                                                  select(-idx) %>%    #remove idx column as it is no longer needed
                                                  replace_na(list(DEMOCRAT=0,LIBERTARIAN=0,OTHER=0,REPUBLICAN=0)) %>%     # replace all NA values in the party columns with 0
                                                  group_by(across(all_of(c("year","state_po")))) %>%    #group by year and state
                                                  summarize(across(everything(), list(sum))) %>%    #summarize everything by adding the values
                                                  ungroup()  # ungroup to create tibble
#rename all columns
names(all_election_result_df) <- c('YEAR', 'STATE', 'DEMOCRAT', 'LIBERTARIAN', 'OTHER', 'REPUBLICAN')
#since OTHER and LIBERTARIAN do not have a high percentage of votes, we can choose between DEMOCRATS and REPUBLICANS as to who has won the election in that state
summary(all_election_result_df)
all_election_result_df$RULING.PARTY <- ifelse(all_election_result_df$DEMOCRAT>all_election_result_df$REPUBLICAN, "DEMOCRAT","REPUBLICAN")
#drop unnecessary and redundant columns
all_election_result_df <- all_election_result_df %>% select(YEAR, STATE, RULING.PARTY)
#only store the records of year 2020 for further analysis
ruling_govt_df <- all_election_result_df %>% filter(YEAR == 2020)
#dimensions of the dataframe will be [51 - rows, 3 columns]. This includes DC
dim(ruling_govt_df)

print("---------- US States and their Regions ----------")
#Get all US states and their Regions
regions_df <- read_csv(url(state_regions_url))
#rename all columns
names(regions_df) <- c('STATE.NAME','STATE', 'REGION', 'DIVISION')
#dimensions of the dataframe will be [51 - rows, 4 columns]. This includes DC
dim(regions_df)

[1] "---------- Electricity price - cents per kilo watt hour ----------"
Rows: 4605 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): State, Industry Sector Category
dbl (7): Year, Residential, Commercial, Industrial, Transportation, Other, T...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "---------- Electricity Generation by type(Sustainable,Not Sustainable) measured in Mega Watt hours ----------"
Rows: 461042 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): STATE, TYPE_OF_PRODUCER, ENERGY_SOURCE
dbl (3): YEAR, MONTH, GENERATION_MWh

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
`summarise()` has grouped output by 'YEAR', 'STATE'. You 

#### Combine dataframes

In [6]:
%%R
str(generation_df)

tibble [104 × 4] (S3: tbl_df/tbl/data.frame)
 $ YEAR             : num [1:104] 2020 2020 2020 2020 2020 2020 2020 2020 2020 2020 ...
 $ STATE            : chr [1:104] "AK" "AK" "AL" "AL" ...
 $ ENERGY.SOURCE    : chr [1:104] "Not Sustainable" "Sustainable" "Not Sustainable" "Sustainable" ...
 $ GENERATION_MM.MWh: num [1:104] 0.142 0.381 2.084 9.378 1.36 ...


In [7]:
%%R
str(consumption_df)

tibble [104 × 5] (S3: tbl_df/tbl/data.frame)
 $ YEAR.tminus1              : num [1:104] 2019 2019 2019 2019 2019 ...
 $ STATE                     : chr [1:104] "AK" "AK" "AL" "AL" ...
 $ ENERGY.SOURCE             : chr [1:104] "Not Sustainable" "Sustainable" "Not Sustainable" "Sustainable" ...
 $ CONSUMPTION_MM.MWh.tminus1: num [1:104] 0.105 15.285 2.684 256.407 2.594 ...
 $ YEAR                      : num [1:104] 2020 2020 2020 2020 2020 2020 2020 2020 2020 2020 ...


In [8]:
%%R
str(emission_df)

tibble [104 × 4] (S3: tbl_df/tbl/data.frame)
 $ YEAR                       : num [1:104] 2020 2020 2020 2020 2020 2020 2020 2020 2020 2020 ...
 $ STATE                      : chr [1:104] "AK" "AK" "AL" "AL" ...
 $ ENERGY.SOURCE              : chr [1:104] "Not Sustainable" "Sustainable" "Not Sustainable" "Sustainable" ...
 $ CO2.EMISSION_MM.METRIC.TONS: num [1:104] 2.21 1.25 21.16 23.64 15.56 ...


In [9]:
%%R
str(ruling_govt_df)

tibble [51 × 3] (S3: tbl_df/tbl/data.frame)
 $ YEAR        : num [1:51] 2020 2020 2020 2020 2020 2020 2020 2020 2020 2020 ...
 $ STATE       : chr [1:51] "AK" "AL" "AR" "AZ" ...
 $ RULING.PARTY: chr [1:51] "REPUBLICAN" "REPUBLICAN" "REPUBLICAN" "DEMOCRAT" ...


In [10]:
%%R
str(regions_df)

spec_tbl_df [51 × 4] (S3: spec_tbl_df/tbl_df/tbl/data.frame)
 $ STATE.NAME: chr [1:51] "Alaska" "Alabama" "Arkansas" "Arizona" ...
 $ STATE     : chr [1:51] "AK" "AL" "AR" "AZ" ...
 $ REGION    : chr [1:51] "West" "South" "South" "West" ...
 $ DIVISION  : chr [1:51] "Pacific" "East South Central" "West South Central" "Mountain" ...
 - attr(*, "spec")=
  .. cols(
  ..   State = col_character(),
  ..   `State Code` = col_character(),
  ..   Region = col_character(),
  ..   Division = col_character()
  .. )
 - attr(*, "problems")=<externalptr> 


In [11]:
%%R
str(price_df)

tibble [52 × 3] (S3: tbl_df/tbl/data.frame)
 $ YEAR            : num [1:52] 2020 2020 2020 2020 2020 2020 2020 2020 2020 2020 ...
 $ STATE           : chr [1:52] "AK" "AL" "AR" "AZ" ...
 $ PRICE.DOLLAR_MWh: num [1:52] 198.2 98.4 83.2 104.4 180 ...


Since price has only one record per state, and so far I have not figured out how to include this separately for Sustainable vs Not Sustainable Energy Sources, I am not merging this into the final dataframe

In [12]:
%%R
#combine dataframes - ruling_govt_df, redions_df, generation_df, consumption_df and emission_df
combined_df <-merge(merge(merge(merge(generation_df, consumption_df, by=c("YEAR","STATE","ENERGY.SOURCE")), emission_df, by=c("YEAR","STATE","ENERGY.SOURCE")), ruling_govt_df, by=c("YEAR","STATE")), regions_df, by=c("STATE"))
combined_df <- combined_df %>% relocate(any_of(c("STATE.NAME", "REGION", "DIVISION", "RULING.PARTY")), .after=STATE) %>% select(,-c("YEAR","YEAR.tminus1"))
dim(combined_df)

[1] 102   9


In [13]:
%R -o combined_df
from IPython.display import display, HTML

display(HTML(combined_df.to_html()))

,STATE,STATE.NAME,REGION,DIVISION,RULING.PARTY,ENERGY.SOURCE,GENERATION_MM.MWh,CONSUMPTION_MM.MWh.tminus1,CO2.EMISSION_MM.METRIC.TONS
1,AK,Alaska,West,Pacific,REPUBLICAN,Not Sustainable,0.1423,0.1052,2.2050
2,AK,Alaska,West,Pacific,REPUBLICAN,Sustainable,0.3810,15.2853,1.2539
3,AL,Alabama,South,East South Central,REPUBLICAN,Not Sustainable,2.0843,2.6837,21.1638
4,AL,Alabama,South,East South Central,REPUBLICAN,Sustainable,9.3776,256.4072,23.6394
5,AR,Arkansas,South,West South Central,REPUBLICAN,Not Sustainable,1.3599,2.5936,15.5566
6,AR,Arkansas,South,West South Central,REPUBLICAN,Sustainable,3.1931,96.5304,7.8969
7,AZ,Arizona,West,Mountain,DEMOCRAT,Not Sustainable,1.1657,2.4248,14.4691
8,AZ,Arizona,West,Mountain,DEMOCRAT,Sustainable,7.9433,220.9166,21.2452
9,CA,California,West,Pacific,DEMOCRAT,Not Sustainable,0.4081,0.0092,1.2938
10,CA,California,West,Pacific,DEMOCRAT,Sustainable,15.6236,381.0005,41.8743


#### Save combined_df as csv to upload into github for later

In [14]:
from google.colab import files

#convert R data fram to python dataframe
%R -o combined_df
combined_df.to_csv('electricity_analysis.csv', encoding = 'utf-8-sig') 
files.download('electricity_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Session Info

In [15]:
%%R
sessionInfo()

R version 4.2.0 (2022-04-22)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas/liblapack.so.3

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] magrittr_2.0.3  forcats_0.5.1   stringr_1.4.0   purrr_0.3.4    
 [5] tidyr_1.2.0     tibble_3.1.6    ggplot2_3.3.5   tidyverse_1.3.1
 [9] dplyr_1.0.9     readr_2.1.2    

loaded via a namespace (and not attached):
 [1] cellranger_1.1.0 pillar_1.7.0   